In [1]:
import tensorflow as tf
import numpy as np
import os
from random import shuffle
from tqdm import tqdm
import cv2

In [2]:
path = ''
IMG_SIZE = 110

In [3]:
def create_data(kind):
    data = []
    main_dir = os.path.join(path, kind)
    cats_dir = os.path.join(main_dir, 'cats')
    dogs_dir = os.path.join(main_dir, 'dogs')
    for img in tqdm(os.listdir(cats_dir)):
            pathc = os.path.join(cats_dir, img)
            img = cv2.imread(pathc, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE)) / 255.0
            data.append([np.array(img), 0])
    for img in tqdm(os.listdir(dogs_dir)):
            pathd = os.path.join(dogs_dir, img)
            img = cv2.imread(pathd, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))/ 255.0
            data.append([np.array(img), 1])
    shuffle(data)
    return data



In [4]:
whole_data = create_data('train')
valid_data = create_data('valid')
training_data = whole_data[:18000]
test_data = whole_data[18000:]


100%|██████████| 1000/1000 [00:12<00:00, 79.65it/s]


In [5]:
X_train = []
y_train = []
X_val = []
y_val = []
for feature, label in training_data:
    X_train.append(feature)
    y_train.append(label)
for feature, label in valid_data:
    X_val.append(feature)
    y_val.append(label) 

In [6]:
from tensorflow.keras import utils
X_train = np.array(X_train).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_train = np.array(y_train)
y_train = utils.to_categorical(y_train, 2)
X_val = np.array(X_val).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_val = np.array(y_val)
y_val = utils.to_categorical(y_val, 2)

In [7]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import regularizers
from tensorflow.keras import initializers
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

In [8]:
model = Sequential([
    Conv2D(16, (3, 3), padding='same', activation='relu', input_shape=(X_train.shape[1:])),
    Dropout(0.25),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    Conv2D(32, (5, 5), padding='same', activation='relu'),
    Conv2D(64, (7, 7), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    Conv2D(128, (9, 9), padding='same', activation='relu'),
    MaxPooling2D(pool_size=(2, 2), strides=(2,2)),
    Flatten(),
    Dropout(0.25),
    Dense(512, activation='relu', kernel_initializer='he_uniform', kernel_regularizer=regularizers.l2(0.0001), bias_regularizer=regularizers.l2(0.0001)),
    Dense(2, activation='softmax')
   
])

In [9]:
adam = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(optimizer=adam,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [10]:
history = model.fit(X_train, y_train, epochs=9, batch_size=64, validation_data=(X_val, y_val))

Train on 18000 samples, validate on 2000 samples
Epoch 1/9
18000/18000 [==============================] - 132s 7ms/sample - loss: 0.6907 - accuracy: 0.6294 - val_loss: 0.6733 - val_accuracy: 0.6395
Epoch 2/9
18000/18000 [==============================] - 90s 5ms/sample - loss: 0.5446 - accuracy: 0.7529 - val_loss: 0.5379 - val_accuracy: 0.7530
Epoch 3/9
18000/18000 [==============================] - 89s 5ms/sample - loss: 0.4728 - accuracy: 0.7938 - val_loss: 0.4725 - val_accuracy: 0.7815
Epoch 4/9
18000/18000 [==============================] - 90s 5ms/sample - loss: 0.4331 - accuracy: 0.8155 - val_loss: 0.4562 - val_accuracy: 0.7920
Epoch 5/9
18000/18000 [==============================] - 90s 5ms/sample - loss: 0.3928 - accuracy: 0.8324 - val_loss: 0.4233 - val_accuracy: 0.8105
Epoch 6/9
18000/18000 [==============================] - 84s 5ms/sample - loss: 0.3528 - accuracy: 0.8566 - val_loss: 0.4604 - val_accuracy: 0.7880
Epoch 7/9
18000/18000 [==============================] - 78s 4

In [13]:
X_test = []
y_test = []
for feature, label in test_data:
    X_test.append(feature)
    y_test.append(label)

X_test = np.array(X_test).reshape(-1, IMG_SIZE, IMG_SIZE, 1)
y_test = np.array(y_test)
y_test = utils.to_categorical(y_test, 2)

In [14]:
results = model.evaluate(X_test, y_test, batch_size=64)
print('test loss, test acc:', results)

5000/5000 [==============================] - 6s 1ms/sample - loss: 0.3750 - accuracy: 0.8440
test loss, test acc: [0.3749985071182251, 0.844]
